In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [33]:
df = pd.read_parquet('data/fhv_tripdata_2021-01.parquet')
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


## Q1

In [34]:
len(df)

1154112

## Q2

In [35]:
df['duration'] = df['dropOff_datetime'] - df['pickup_datetime']
df['duration'].mean().total_seconds()/60

19.167224083333334

## Q3

In [36]:
# Data cleaning
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [37]:
categorical = ['PUlocationID', 'DOlocationID']
df[categorical] = df[categorical].fillna(-1)

In [38]:
sum(df['PUlocationID'] == -1)/len(df)

0.8352732770722617

## Q4

In [39]:
# Data cleaning
df[categorical] = df[categorical].astype(str)

In [14]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [20]:
X_train.shape[1]

525

## Q5

In [17]:
y_train = df['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107206316

## Q6

In [21]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df['dropOff_datetime'] - df['pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].fillna(-1)
    df[categorical] = df[categorical].astype(str)
    
    return df

In [22]:
df_train = read_dataframe('data/fhv_tripdata_2021-01.parquet')
df_val = read_dataframe('data/fhv_tripdata_2021-02.parquet')

In [23]:
categorical = ['PUlocationID', 'DOlocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [24]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [25]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.014283149347039